# Thresholding and noise

In this exercise, we will try out different threshold methods, see how they react to noisy data - and how filtering can help to improve the results. To finish this excercise, have a look at the notebook's from [last week]('../04_image_processing_and_filters') - you may find some useful code!

In [1]:
from skimage import data, filters, measure, util, morphology
import matplotlib.pyplot as plt
import napari
import numpy as np

## About pixel noise...

Pixel noise can have very different characteristics - each of them requires a different type of image filtering to get rid of it. When we talk about noise, we typically differentiate between pixel noise and image noise, both of which have different origins and some subtypes. 

**Pixel noise**: This can, for instance, originate from electronic noise of the camera at the microscope and influences the pixels of the image individually.

In [13]:
data = np.zeros((5,5))
data

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

There are different types of pixel noise. *Gaussian noise*, for instance, adds a small value to every pixel according to a gaussian distribution. The gaussian filter provides a good way to deal with this.

In [14]:
data_gaussian_noise = util.random_noise(data, mode='gaussian')
data_gaussian_noise

array([[0.        , 0.        , 0.11628047, 0.01184722, 0.05113202],
       [0.        , 0.13078862, 0.        , 0.        , 0.03313759],
       [0.        , 0.12232928, 0.        , 0.15295342, 0.06675278],
       [0.02888014, 0.1172457 , 0.1052611 , 0.11746526, 0.01738648],
       [0.06713701, 0.05122907, 0.        , 0.12834537, 0.        ]])

*Salt noise* changes random pixels to high values - such noise can originate from dead pixels of the detector hardware. The median filter provides a good way to deal with this

In [23]:
data_salt_noise = util.random_noise(data, mode='salt')
data_salt_noise

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

Conversely, *pepper noise* can change single pixels to very low values, which can also originate from hardware isues. The median filter also provides a good option to deal with this type of noise.

In [26]:
data = np.ones((5,5))
data_pepper_noise = util.random_noise(data, mode='pepper')
data_pepper_noise

array([[1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

The image background represents a type of noise that affects large areas of the image, which can orginate from heterogeneous lighting at a microscope or autofluorescence of fluorescent markers. The top-hat filter provides a suitable means to address this problem.

## Excercises

First, we create some data and then artifically add different kinds of noise to it - and see how we can best remove these further down the line again to make our downstream analysis more robust.

In [ ]:
# First, we create some noisy data
image = data.human_mitosis()
plt.imshow(image, cmap='gray')

## Exercise 1

Let's add some pixel noise to the data. Skimage provides some [handy tools](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=utils#skimage.util.random_noise) for this. Add or remove some of the noise contributions by un-commenting some of the lines below - you can also add multiple sources of noise to the image.

*Hint*: Use the `vmin` and `vmax` parameters to better visualize the brightness and contrast.

In [ ]:
noisy_data = util.random_noise(image, mode='gaussian') * 255
#noisy_data = utils.random_noise(image, mode='pepper') * 255
#noisy_data = utils.random_noise(image, mode='s&p') * 255
#noisy_data = utils.random_noise(image, mode='salt') * 255
plt.imshow(noisy_data, cmap='gray', vmin=0, vmax=255)

## Exercise 2

Now, we add another type of noise - background - and add it to the noisy data from above.

*Optional*: Try to understand the code below. What happens when you modify the value for `sigma` in line 3?

In [ ]:
bg = np.zeros_like(noisy_data)
bg[50,50] = 255
bg_blur = filters.gaussian(bg, sigma=300)
bg_blur = 255*bg_blur/bg_blur.max()
noisy_data += bg_blur
plt.imshow(noisy_data, cmap='gray')  # adjust the values for vmin and vmax to get a better impression!

## Exercise 3:

In order to clear the added noise to the raw image in variable `noisy_data`, you need to apply **noise removal** and **background subtraction**. 

*Hint 1*: The `morphology.white_tophat()` and `filters.gaussian()` may provide the correct means to remove some types of noise in the image - but make sure to pass suitable parameters to these functions!

- background subtraction with `morphology.white_tophat()`: Have a look at the footprint parameter!
- Noise removal with `filters.gaussian()`: Have a look at the sigma parameters!

*Hint 2*: Look up the [documentation](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=utils#skimage.util.random_noise) for `salt & pepper` noise - how are pixel values changed if this type of noise is applied? Which filter operation could be suitable to deal with single pixels with high grey values? 

In [ ]:
footprint = morphology.disk(7)
background_subtracted = morphology.white_tophat(noisy_data, footprint=footprint)
plt.imshow(background_subtracted, cmap='gray')

In [ ]:
clean_image = filters.gaussian(background_subtracted, sigma=2)
plt.imshow(clean_image, cmap='gray')

## Exercise 4

Now, apply thresholds from `skimage` to the image data! Hint: You can find them under `filters.threshold_...` - use the `tab` key or the [documentation](https://scikit-image.org/docs/stable/api/skimage.filters.html?highlight=filters#module-skimage.filters) to find out how to use the implemented threshold functions.

In [ ]:
binary = filters.threshold_otsu

## Exercise 5

Lastly, perform a connected-component-analysis to create a label image from the previously generated binary image with `measure.label()`. 

Use both `plt.imshow()` and Napari (`viewer.add_labels()`) to visualize the label image - why do the results *look* completely different?

In [ ]:
viewer = napari.Viewer()